In [1]:
import pandas as pd
import requests
import os

In [2]:
df = pd.read_csv('data.csv')
df.head()

,id,name,schedule,accept_temporary,published_at,city,salary_mean
0,74376536,Data scientist,remote,False,2022-12-29T12:15:51+0300,Москва,175000.0
1,74157498,Data scientist,fullDay,False,2022-12-22T17:50:42+0300,Москва,170000.0
2,54372021,TechLead Research Engineer (NLP/Speech),remote,False,2022-12-23T10:07:53+0300,Москва,350000.0
3,54372020,TechLead Research Engineer (NLP/Speech),remote,False,2022-12-23T10:07:50+0300,Санкт-Петербург,350000.0
4,54372019,TechLead Research Engineer (NLP/Speech),remote,False,2022-12-23T10:07:47+0300,Иннополис,350000.0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                33 non-null     int64  
 1   name              33 non-null     object 
 2   schedule          33 non-null     object 
 3   accept_temporary  33 non-null     bool   
 4   published_at      33 non-null     object 
 5   city              33 non-null     object 
 6   salary_mean       33 non-null     float64
dtypes: bool(1), float64(1), int64(1), object(4)
memory usage: 1.7+ KB


In [18]:
features = {
    'skills': [],
    'experience': [],
    'professional roles': [],
    'employer trusted': [],
    'url':[],
}

for indx, id in enumerate(df['id']):
    os.system('clear')    
    req = requests.get(f'https://api.hh.ru/vacancies/{id}').json()
    skills = [skill['name'] for skill in req['key_skills'] if isinstance(req, dict)]
    features['skills'].append(skills)
    features['experience'].append(req['experience']['name'])
    roles = [role['name'] for role in req['professional_roles'] if isinstance(req, dict)]
    features['professional roles'].append(roles)
    features['employer trusted'].append(req['employer']['trusted'])
    features['url'].append(req['alternate_url'])
    print(f'skill parcing: {(indx+1) * 100/len(df):.2f}% comleted')

skill parcing: 3.03% comleted
skill parcing: 6.06% comleted
skill parcing: 9.09% comleted
skill parcing: 12.12% comleted
skill parcing: 15.15% comleted
skill parcing: 18.18% comleted
skill parcing: 21.21% comleted
skill parcing: 24.24% comleted
skill parcing: 27.27% comleted
skill parcing: 30.30% comleted
skill parcing: 33.33% comleted
skill parcing: 36.36% comleted
skill parcing: 39.39% comleted
skill parcing: 42.42% comleted
skill parcing: 45.45% comleted
skill parcing: 48.48% comleted
skill parcing: 51.52% comleted
skill parcing: 54.55% comleted
skill parcing: 57.58% comleted
skill parcing: 60.61% comleted
skill parcing: 63.64% comleted
skill parcing: 66.67% comleted
skill parcing: 69.70% comleted
skill parcing: 72.73% comleted
skill parcing: 75.76% comleted
skill parcing: 78.79% comleted
skill parcing: 81.82% comleted
skill parcing: 84.85% comleted
skill parcing: 87.88% comleted
skill parcing: 90.91% comleted
skill parcing: 93.94% comleted
skill parcing: 96.97% comleted
skill parci

In [19]:
features

{'skills': [['Python', 'PyTorch', 'Machine Learning', 'Spark', 'SQL'],
  ['Python',
   'Работа с базами данных',
   'MS PowerPoint',
   'Работа с большим объемом информации',
   'VBA'],
  ['Python', 'PyTorch', 'NLP', 'SQL', 'Data Science', 'Speech', 'Ner'],
  ['Python', 'PyTorch', 'NLP', 'SQL', 'Data Science', 'Speech', 'Ner'],
  ['Python', 'PyTorch', 'NLP', 'SQL', 'Data Science', 'Speech', 'Ner'],
  ['Python', 'PyTorch', 'Deep Learning', 'Git', 'OpenCV'],
  ['Python', 'Git', 'Английский язык', 'Atlassian Jira', 'NLP'],
  ['Python',
   'PyTorch',
   'SQL',
   'Machine Learning',
   'Анализ данных',
   'data science',
   'numpy',
   'scipy',
   'pandas',
   'lightgbm',
   'pytorch',
   'scikit-learn',
   'neural nets',
   'nlp',
   'search ranking',
   'recommender systems',
   'a/b-experiments',
   'hadoop',
   'hive',
   'presto',
   'Mathematics'],
  ['Python', 'Git', 'Linux', 'Английский язык', 'PyTorch'],
  ['Python', 'PyTorch', 'NLP', 'Numpy', 'Scikit-learn'],
  ['Python', 'Machin